# Вариант 5

In [ ]:
%%writefile roberts.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

#define CSC(call)                                    \
do {                                          \
    cudaError_t res = call;                              \
    if (res != cudaSuccess) {                          \
        fprintf(stderr, "ERROR in %s:%d. Message: %s\n",      \
                __FILE__, __LINE__, cudaGetErrorString(res));    \
        exit(0);                                    \
    }                                            \
} while(0)

// Локальная функция для вычисления яркости
__device__ double luminance(uchar4 p) {
    double red = p.x;
    double green = p.y;
    double blue = p.z;
    return (0.299 * red + 0.587 * green + 0.114 * blue);
}

__global__ void kernel(cudaTextureObject_t tex, uchar4 *out, int w, int h) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    int idy = blockDim.y * blockIdx.y + threadIdx.y;
    int offsetx = blockDim.x * gridDim.x;
    int offsety = blockDim.y * gridDim.y;

    uchar4 w11, w12, w21, w22;
    double Gx, Gy, gradientLength;

    for (int y = idy; y < h; y += offsety) {
        for (int x = idx; x < w; x += offsetx) {
            // Получаем значения пикселей
            w11 = tex2D<uchar4>(tex, x, y);
            w12 = tex2D<uchar4>(tex, (x + 1), y);
            w21 = tex2D<uchar4>(tex, x, (y + 1));
            w22 = tex2D<uchar4>(tex, (x + 1), (y + 1));

            // Вычисляем градиенты по методу Робертса
            Gx = luminance(w22) - luminance(w11);
            Gy = luminance(w21) - luminance(w12);

            // Вычисляем длину градиента
            gradientLength = sqrt(Gx * Gx + Gy * Gy);
            float answer = min(255.0f, (float)round(gradientLength));

            // Сохраняем результат в выходной массив
            out[y * w + x] = make_uchar4(answer, answer, answer, w11.w);
        }
    }
}

int main() {
    char input_file[100], output_file[100];

    // Считываем имена входного и выходного файлов
    scanf("%s", input_file);
    scanf("%s", output_file);

    // Открываем входной файл
    FILE *fp = fopen(input_file, "rb");
    if (fp == NULL) {
        fprintf(stderr, "Ошибка открытия файла %s\n", input_file);
        return -1;
    }

    // Считываем ширину и высоту изображения
    int w, h;
    fread(&w, sizeof(int), 1, fp);
    fread(&h, sizeof(int), 1, fp);

    // Считываем пиксели
    uchar4 *data = (uchar4 *)malloc(sizeof(uchar4) * w * h);
    fread(data, sizeof(uchar4), w * h, fp);
    fclose(fp);

    // Выделяем память на устройстве для текстуры
    cudaArray *arr;
    cudaChannelFormatDesc ch = cudaCreateChannelDesc<uchar4>();
    CSC(cudaMallocArray(&arr, &ch, w, h));

    // Копируем данные в текстуру
    CSC(cudaMemcpy2DToArray(arr, 0, 0, data, w * sizeof(uchar4), w * sizeof(uchar4), h, cudaMemcpyHostToDevice));

    // Настройка текстурного объекта
    struct cudaResourceDesc resDesc;
    memset(&resDesc, 0, sizeof(resDesc));
    resDesc.resType = cudaResourceTypeArray;
    resDesc.res.array.array = arr;

    struct cudaTextureDesc texDesc;
    memset(&texDesc, 0, sizeof(texDesc));
    texDesc.addressMode[0] = cudaAddressModeClamp;
    texDesc.addressMode[1] = cudaAddressModeClamp;
    texDesc.filterMode = cudaFilterModePoint;
    texDesc.readMode = cudaReadModeElementType;
    texDesc.normalizedCoords = false;

    cudaTextureObject_t tex = 0;
    CSC(cudaCreateTextureObject(&tex, &resDesc, &texDesc, NULL));

    // Выделяем память для выходных данных на устройстве
    uchar4 *dev_out;
    CSC(cudaMalloc(&dev_out, sizeof(uchar4) * w * h));

    // Запускаем ядро Робертса
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((w + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (h + threadsPerBlock.y - 1) / threadsPerBlock.y);
    kernel<<<numBlocks, threadsPerBlock>>>(tex, dev_out, w, h);
    CSC(cudaGetLastError());

    // Копируем результат обратно на хост
    CSC(cudaMemcpy(data, dev_out, sizeof(uchar4) * w * h, cudaMemcpyDeviceToHost));

    // Освобождаем текстурный объект и память на устройстве
    CSC(cudaDestroyTextureObject(tex));
    CSC(cudaFreeArray(arr));
    CSC(cudaFree(dev_out));

    // Открываем выходной файл
    fp = fopen(output_file, "wb");
    if (fp == NULL) {
        fprintf(stderr, "Ошибка открытия файла %s\n", output_file);
        free(data);
        return -1;
    }

    // Записываем размеры и данные в выходной файл
    fwrite(&w, sizeof(int), 1, fp);
    fwrite(&h, sizeof(int), 1, fp);
    fwrite(data, sizeof(uchar4), w * h, fp);
    fclose(fp);

    // Освобождаем память
    free(data);
    return 0;
}

Overwriting roberts.cu


In [ ]:
!nvcc -o roberts roberts.cu

In [ ]:
!./roberts

in2.data
out2.data


In [ ]:
# Открываем бинарный файл и читаем его содержимое
with open('out2.data', 'rb') as binary_file:
    binary_data = binary_file.read()

# Преобразуем бинарные данные в строку hex
hex_data = binary_data.hex()

# Форматируем строку hex для удобства (например, с пробелами каждые 8 символов)
formatted_hex = ' '.join([hex_data[i:i+8] for i in range(0, len(hex_data), 8)])

# Выводим полученный результат
print(formatted_hex)

03000000 03000000 80808000 80808000 00000000 80808000 80808000 00000000 00000000 00000000 00000000
